In [2]:
import tensorflow as tf

import numpy as np
import sys
sys.path.append('..')

In [3]:
from models import LocalNoiseRBM


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [4]:
tf.enable_eager_execution()

In [5]:
nv = 10
nh = 2
rbm = LocalNoiseRBM(nv, nh)

In [7]:
v = tf.zeros((nv,1))

In [ ]:
rbm.compute_hidden_probs()